In [1]:
import sys
sys.path.append("/osdrive/home/sktime/sktime/forecasting/model_selection/")
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.datasets import load_longley
from sktime.split import SingleWindowSplitter, SlidingWindowSplitter
from sktime.forecasting.tests._config import (
    # TEST_N_ITERS,
    TEST_OOS_FHS,
    # TEST_RANDOM_SEEDS,
    # TEST_WINDOW_LENGTHS_INT,
)
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.impute import Imputer
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
   TuneForecastingOptunaCV,
)
import pandas as pd
import sktime.forecasting.model_selection
import optuna

pd.set_option("display.max_colwidth", None)

CVs = [
    *[SingleWindowSplitter(fh=fh) for fh in TEST_OOS_FHS],
    SlidingWindowSplitter(fh=1, initial_window=12, step_length=3),
]

# params_distributions = {
#     "forecaster": [NaiveForecaster()],
#     "forecaster__window_length": [1, 5],
#     "forecaster__strategy": ["drift", "last", "mean"],
#     "imputer__method": ["mean", "median"],
# }

params_distributions = {
    #"forecaster": [NaiveForecaster()],
    "forecaster__window_length": optuna.distributions.IntDistribution(1, 5),
    "forecaster__strategy": optuna.distributions.CategoricalDistribution(["drift", "last", "mean"]),
    "imputer__method": optuna.distributions.CategoricalDistribution(["mean", "median"]),
}


cv = CVs[-1]
y, X = load_longley()
pipe = TransformedTargetForecaster(
    steps=[("imputer", Imputer()), ("forecaster", NaiveForecaster())]
)
sscv = TuneForecastingOptunaCV(
    forecaster=pipe,
    param_grid=params_distributions,
    cv=cv,
    n_evals=5,

)
sscv.fit(y, X)
print(sscv.cv_results_.params)
print(cv)


/home/m86149/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-06-11 07:52:36,496] A new study created in memory with name: no-name-8f3baefa-29f6-4743-9c86-8c87cd02478d
[I 2024-06-11 07:52:36,560] Trial 0 finished with value: 0.03087350235905039 and parameters: {}. Best is trial 0 with value: 0.03087350235905039.
/osdrive/home/sktime/sktime/utils/parallel.py:92: FitFailedWarning: 
                In evaluate, fitting of forecaster TransformedTargetForecaster failed,
                you can set error_score='raise' in evaluate to see
                the exception message.
                Fit failed for the 0-th data split, on training data y_train with
                cutoff NaT, and len(y_train)=12.
                The score will be set to nan.
                Failed forecaster with paramete

{'forecaster__window_length': 5, 'forecaster__strategy': 'mean', 'imputer__method': 'median'}
{'forecaster__window_length': 1, 'forecaster__strategy': 'drift', 'imputer__method': 'median'}
{'forecaster__window_length': 5, 'forecaster__strategy': 'last', 'imputer__method': 'mean'}
{'forecaster__window_length': 4, 'forecaster__strategy': 'mean', 'imputer__method': 'median'}


[I 2024-06-11 07:52:36,743] Trial 6 finished with value: 0.025461032212043377 and parameters: {}. Best is trial 4 with value: 0.024245965371232633.
[I 2024-06-11 07:52:36,819] Trial 8 finished with value: 0.024245965371232633 and parameters: {}. Best is trial 4 with value: 0.024245965371232633.


{'forecaster__window_length': 1, 'forecaster__strategy': 'last', 'imputer__method': 'mean'}
0                                                                                                {}
1     {'forecaster__window_length': 5, 'forecaster__strategy': 'mean', 'imputer__method': 'median'}
2                                                                                                {}
3    {'forecaster__window_length': 1, 'forecaster__strategy': 'drift', 'imputer__method': 'median'}
4                                                                                                {}
5       {'forecaster__window_length': 5, 'forecaster__strategy': 'last', 'imputer__method': 'mean'}
6                                                                                                {}
7     {'forecaster__window_length': 4, 'forecaster__strategy': 'mean', 'imputer__method': 'median'}
8                                                                                                {}
9       

{'forecaster': NaiveForecaster(), 'forecaster__window_length': 1, 'forecaster__strategy': 'drift', 'imputer__method': 'median'}
{'forecaster': NaiveForecaster(strategy='drift', window_length=1), 'forecaster__window_length': 5, 'forecaster__strategy': 'last', 'imputer__method': 'median'}
{'forecaster': NaiveForecaster(window_length=5), 'forecaster__window_length': 1, 'forecaster__strategy': 'mean', 'imputer__method': 'mean'}
{'forecaster': NaiveForecaster(strategy='mean', window_length=1), 'forecaster__window_length': 5, 'forecaster__strategy': 'last', 'imputer__method': 'mean'}


In [ ]:
type(params_distributions)
for name, v in params_distributions.items():
    print(name, v)

In [ ]:
import optuna
from sklearn.datasets import load_iris
from sklearn.svm import SVC

clf = SVC(gamma="auto")
param_distributions = {
    "C": optuna.distributions.FloatDistribution(1e-10, 1e10, log=True)
}
optuna_search = optuna.integration.OptunaSearchCV(clf, param_distributions)
X, y = load_iris(return_X_y=True)
optuna_search.fit(X, y)
y_pred = optuna_search.predict(X)

In [ ]:
import optuna
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the classifier
clf = SVC(gamma="auto")

# Define the parameter distributions for Optuna
param_distributions = {
    "C": optuna.distributions.FloatDistribution(1e-10, 1e10, log=True)
}

# Create the OptunaSearchCV
optuna_search = optuna.integration.OptunaSearchCV(clf, param_distributions)

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # normalize data
    ('classifier', optuna_search)  # apply classifier
])

# Load the data
X, y = load_iris(return_X_y=True)

# Fit the pipeline
pipeline.fit(X, y)

# Make a prediction
y_pred = pipeline.predict(X)

In [ ]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sktime.datasets import load_longley

# Define the parameter distributions for Optuna
param_distributions = {
    "n_estimators": optuna.distributions.IntDistribution(10, 100),
    "max_depth": optuna.distributions.IntDistribution(1, 10),
}

from sklearn.model_selection import TimeSeriesSplit

# Define the number of splits
n_splits = 5

# Create the TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=n_splits)

# Create the OptunaSearchCV
optuna_search = optuna.integration.OptunaSearchCV(
    RandomForestRegressor(), 
    param_distributions, 
    cv=tscv, 
    n_trials=10
)

# Define the pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # impute missing values
    ('scaler', StandardScaler()),  # normalize data
    ('regressor', optuna_search)  # apply regressor
])

# Load the data
y, X = load_longley()

# Fit the pipeline
pipeline.fit(X, y)

# Make a prediction
y_pred = pipeline.predict(X)